In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from ast import literal_eval

c:\Users\zer0\Documents\learn\embeddings\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
df = pd.read_csv('data/25k-imdb-movie-dataset.csv')
df.head()

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/
3,Lightyear,"$71,101,257",5.2,32K,"['Animation', 'Action', 'Adventure']",While spending years attempting to return home...,"['galaxy', 'spaceship', 'robot', 'rocket', 'sp...",Angus MacLane,"['Jason Headley', 'Matthew Aldrich', 'Chris Ev...",Angus MacLane,-2022,/title/tt10298810/
4,Spiderhead,not-released,5.4,23K,"['Action', 'Crime', 'Drama']","In the near future, convicts are offered the c...","['discover', 'medical', 'test', 'reality', 'fi...",Joseph Kosinski,"['Rhett Reese', 'Paul Wernick', 'Chris Hemswor...",George Saunders,-2022,/title/tt9783600/


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24402 entries, 0 to 24401
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie title   24402 non-null  object
 1   Run Time      24402 non-null  object
 2   Rating        24402 non-null  object
 3   User Rating   24402 non-null  object
 4   Generes       24402 non-null  object
 5   Overview      24158 non-null  object
 6   Plot Kyeword  24402 non-null  object
 7   Director      24402 non-null  object
 8   Top 5 Casts   24402 non-null  object
 9   Writer        24402 non-null  object
 10  year          23624 non-null  object
 11  path          24402 non-null  object
dtypes: object(12)
memory usage: 2.2+ MB


In [4]:
def concate_list(lista: list):
    lista = literal_eval(lista)
    return ' '.join(lista)

def str2list(string: str):
    return literal_eval(string)

In [5]:
df = df.fillna(' ')
df['Keywords'] = df['Plot Kyeword'].apply(concate_list)
df['Starts'] = df['Top 5 Casts'].apply(concate_list)
df['Generes'] = df['Generes'].apply(str2list)
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').fillna(0).astype('float')

df.head()

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path,Keywords,Starts
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"[Action, Drama]",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/,fighter jet sequel u.s. navy fighter aircraft ...,Jack Epps Jr. Peter Craig Tom Cruise Jennifer ...
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"[Action, Adventure, Sci-Fi]",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/,dinosaur jurassic park tyrannosaurus rex veloc...,Colin Trevorrow Derek Connolly Chris Pratt Bry...
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"[Action, Drama]",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/,pilot male camaraderie u.s. navy grumman f 14 ...,Jack Epps Jr. Ehud Yonay Tom Cruise Tim Robbin...
3,Lightyear,"$71,101,257",5.2,32K,"[Animation, Action, Adventure]",While spending years attempting to return home...,"['galaxy', 'spaceship', 'robot', 'rocket', 'sp...",Angus MacLane,"['Jason Headley', 'Matthew Aldrich', 'Chris Ev...",Angus MacLane,-2022,/title/tt10298810/,galaxy spaceship robot rocket space adventure ...,Jason Headley Matthew Aldrich Chris Evans Keke...
4,Spiderhead,not-released,5.4,23K,"[Action, Crime, Drama]","In the near future, convicts are offered the c...","['discover', 'medical', 'test', 'reality', 'fi...",Joseph Kosinski,"['Rhett Reese', 'Paul Wernick', 'Chris Hemswor...",George Saunders,-2022,/title/tt9783600/,discover medical test reality fictional drug v...,Rhett Reese Paul Wernick Chris Hemsworth Miles...


In [6]:
generes = df['Generes'].explode().unique()
generes

array(['Action', 'Drama', 'Adventure', 'Sci-Fi', 'Animation', 'Crime',
       'Comedy', 'Thriller', 'Fantasy', 'Horror', 'History', 'Mystery',
       'Biography', 'War', 'Western', 'Sport', 'Family', 'Romance',
       'Music', 'Musical', 'Film-Noir', 'Game-Show', 'Adult',
       'Reality-TV'], dtype=object)

In [7]:
df.drop(columns=['Plot Kyeword', 'Top 5 Casts'], inplace=True)

In [8]:
df['text'] = df.apply(lambda x: f"{x['Overview']} {x['Keywords']} {x['Starts']} {' '.join(x['Generes'])}", axis=1)
df.iloc[0]['text']

"After more than thirty years of service as one of the Navy's top aviators, Pete Mitchell is where he belongs, pushing the envelope as a courageous test pilot and dodging the advancement in rank that would ground fighter jet sequel u.s. navy fighter aircraft pilot naval aviation jet fighter military pilot aviation aircraft Jack Epps Jr. Peter Craig Tom Cruise Jennifer Connelly Miles Teller Action Drama"

In [10]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [11]:
embeddings = model.encode(df['text'], batch_size=64, show_progress_bar=True)

Batches: 100%|██████████| 382/382 [28:47<00:00,  4.52s/it]


In [12]:
df['embeddings'] = embeddings.tolist()
df['ids'] = df.index
df['ids'] = df['ids'].astype(str)

In [13]:
df.head()

,movie title,Run Time,Rating,User Rating,Generes,Overview,Director,Writer,year,path,Keywords,Starts,text,embeddings,ids
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"[Action, Drama]",After more than thirty years of service as one...,Joseph Kosinski,Jim Cash,-2022,/title/tt1745960/,fighter jet sequel u.s. navy fighter aircraft ...,Jack Epps Jr. Peter Craig Tom Cruise Jennifer ...,After more than thirty years of service as one...,"[-0.07981297373771667, -0.013015184551477432, ...",0
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"[Action, Adventure, Sci-Fi]",Four years after the destruction of Isla Nubla...,Colin Trevorrow,Emily Carmichael,-2022,/title/tt8041270/,dinosaur jurassic park tyrannosaurus rex veloc...,Colin Trevorrow Derek Connolly Chris Pratt Bry...,Four years after the destruction of Isla Nubla...,"[-0.02374485693871975, -0.06632046401500702, 0...",1
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"[Action, Drama]",As students at the United States Navy's elite ...,Tony Scott,Jim Cash,-1986,/title/tt0092099/,pilot male camaraderie u.s. navy grumman f 14 ...,Jack Epps Jr. Ehud Yonay Tom Cruise Tim Robbin...,As students at the United States Navy's elite ...,"[-0.011591935530304909, 0.012706843204796314, ...",2
3,Lightyear,"$71,101,257",5.2,32K,"[Animation, Action, Adventure]",While spending years attempting to return home...,Angus MacLane,Angus MacLane,-2022,/title/tt10298810/,galaxy spaceship robot rocket space adventure ...,Jason Headley Matthew Aldrich Chris Evans Keke...,While spending years attempting to return home...,"[-0.0636165663599968, -0.008158060722053051, -...",3
4,Spiderhead,not-released,5.4,23K,"[Action, Crime, Drama]","In the near future, convicts are offered the c...",Joseph Kosinski,George Saunders,-2022,/title/tt9783600/,discover medical test reality fictional drug v...,Rhett Reese Paul Wernick Chris Hemsworth Miles...,"In the near future, convicts are offered the c...","[-0.07259418070316315, 0.05531562492251396, 0....",4


In [14]:
df.to_csv('data/25k-imdb-movie-dataset-embeddings.csv', index=True)

## ChromaDB

In [14]:
import chromadb
from chromadb.utils import embedding_functions
from ast import literal_eval
import pandas as pd

In [15]:
df = pd.read_csv('data/25k-imdb-movie-dataset-embeddings.csv')
df['ids'] = df['ids'].astype(str)
df['embeddings'] = df['embeddings'].apply(literal_eval)
df.head(2)

,Unnamed: 0,movie title,Run Time,Rating,User Rating,Generes,Overview,Director,Writer,year,path,Keywords,Starts,text,embeddings,ids
0,0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,Joseph Kosinski,Jim Cash,-2022,/title/tt1745960/,fighter jet sequel u.s. navy fighter aircraft ...,Jack Epps Jr. Peter Craig Tom Cruise Jennifer ...,After more than thirty years of service as one...,"[-0.07981297373771667, -0.013015184551477432, ...",0
1,1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,Colin Trevorrow,Emily Carmichael,-2022,/title/tt8041270/,dinosaur jurassic park tyrannosaurus rex veloc...,Colin Trevorrow Derek Connolly Chris Pratt Bry...,Four years after the destruction of Isla Nubla...,"[-0.02374485693871975, -0.06632046401500702, 0...",1


In [4]:
cohere_ef = embedding_functions.CohereEmbeddingFunction(
    api_key='api_key',
    model_name='model_id',
)

sentence_transformers_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name='all-MiniLM-L6-v2',
)

c:\Users\zer0\Documents\learn\embeddings\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [8]:
chroma_client = chromadb.Client()
client_persistent = chromadb.PersistentClient(path='data/data_embeddings')

In [9]:
try:
    db = client_persistent.create_collection(
        name='imdb_movies',
        embedding_function=sentence_transformers_ef,
    )
except Exception as e:
    db = client_persistent.get_collection('imdb_movies')

In [16]:
for i in range(0, len(df), 1000):
    db.add(
        ids=df['ids'].iloc[i:i+1000].tolist(),
        embeddings=df['embeddings'].iloc[i:i+1000].tolist(),
        metadatas=df.drop(columns=['embeddings', 'ids', 'text', 'Generes'], axis=1).iloc[i:i+1000].to_dict(orient='records'),
    )

# db.add(
#     ids=df['ids'].tolist(),
#     embeddings=df['embeddings'].tolist(),
#     metadatas=df.drop(columns=['embeddings', 'ids', 'text', 'Generes'], axis=1).to_dict(orient='records'),
# )

In [18]:
# db.peek(1)

## ChromaDB embeddinngs

In [19]:
db_no_embedding = client_persistent.create_collection(
    name='imdb_movies_no_embedding',
    embedding_function=sentence_transformers_ef,
)

In [20]:
db_no_embedding.add(
    ids=df['ids'].iloc[:1000].tolist(),
    documents=df['text'].iloc[:1000].tolist(),
    metadatas=df.drop(columns=['embeddings', 'ids', 'text', 'Generes'], axis=1).iloc[:1000].to_dict(orient='records'),
)

In [22]:
# db_no_embedding.peek(1)

## ChromaDB Query

In [23]:
result = db.query(
    query_texts=['a history with elves and orcs'],
    n_results=5,
)

result

C:\Users\zer0\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [02:18<00:00, 600kiB/s] 


{'ids': [['83', '3635', '16159', '755', '17405']],
 'distances': [[1.0240474939346313,
   1.0335839986801147,
   1.070363163948059,
   1.1017255783081055,
   1.1139330863952637]],
 'metadatas': [[{'Director': 'Peter Jackson',
    'Keywords': 'ring quest hobbit middle earth epic elf orc based on novel saving the world wizard',
    'Overview': 'A meek Hobbit from the Shire and eight companions set out on a journey to destroy the powerful One Ring and save Middle-earth from the Dark Lord Sauron.',
    'Rating': 8.8,
    'Run Time': '$93,000,000 (estimated)',
    'Starts': 'Fran Walsh Philippa Boyens Elijah Wood Ian McKellen Orlando Bloom',
    'Unnamed: 0': 83,
    'User Rating': '1.8M',
    'Writer': 'J.R.R. Tolkien',
    'movie title': 'The Lord of the Rings: The Fellowship of the Ring',
    'path': '/title/tt0120737/',
    'year': '-2001'},
   {'Director': 'Kate Madison',
    'Keywords': 'orc middle earth fictional language ranger lord of the rings f rated heir sword and sorcery bloodl

### Where

`{"metadata_field": { <Operator>: <Value> }}`

`$eq`: Equal to (string, int, float)

`$ne`: Not equal to (string, int, float)

`$gt`: Greater than (int, float)

`$gte`: Greater than or equal to (int, float)

`$lt`: Less than (int, float)

`$lte`: Less than or equal to (int, float)

`$and`: Logical AND (list of values)

`$or`: Logical OR (list of values)

`$in`: In (list of values)

`$nin`: Not in (list of values)

`$exists`: Field exists (bool)

`$regex`: Regular expression (string)

`$size`: Size of array (int)

`$type`: Type of field (int)

`$not`: Negation of query (dict)

In [25]:
result_where = db.query(
    query_texts=['a history with elves and orcs'],
    n_results=3,
    where={
        'Rating': {
            '$gte': 8,
        },
    }
)

result_where

{'ids': [['83', '146', '17251']],
 'distances': [[1.0240474939346313, 1.3164366483688354, 1.3708646297454834]],
 'metadatas': [[{'Director': 'Peter Jackson',
    'Keywords': 'ring quest hobbit middle earth epic elf orc based on novel saving the world wizard',
    'Overview': 'A meek Hobbit from the Shire and eight companions set out on a journey to destroy the powerful One Ring and save Middle-earth from the Dark Lord Sauron.',
    'Rating': 8.8,
    'Run Time': '$93,000,000 (estimated)',
    'Starts': 'Fran Walsh Philippa Boyens Elijah Wood Ian McKellen Orlando Bloom',
    'Unnamed: 0': 83,
    'User Rating': '1.8M',
    'Writer': 'J.R.R. Tolkien',
    'movie title': 'The Lord of the Rings: The Fellowship of the Ring',
    'path': '/title/tt0120737/',
    'year': '-2001'},
   {'Director': 'Peter Jackson',
    'Keywords': 'epic orc hobbit ring middle earth battle quest journey wizard good versus evil',
    'Overview': "Gandalf and Aragorn lead the World of Men against Sauron's army to 

In [26]:
db.count()

24402